In [2]:
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import gc
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from keras.utils.vis_utils import plot_model
from sklearn.metrics.pairwise import cosine_similarity
import pickle
tqdm.pandas()

In [ ]:
item_attributes = pd.read_csv("../dataset/dressipi_recsys2022/item_attributes.csv")
item_attributes = item_attributes[item_attributes.columns[:905]]
item_attributes

In [ ]:
len(set(list(item_attributes['item_id'])))

23691

In [ ]:
class VariationalAutoEncoder(keras.Model):
    def __init__(self, input_shape, latent_size):
        super(VariationalAutoEncoder, self).__init__()
        self.input_shape = input_shape
        self.latent_size = latent_size
        self.input_layer = tf.keras.Input(shape=self.input_shape)
        self.encoder_layer = tf.keras.layers.Dense(units=512)(self.input_layer)
        self.encoder_layer = tf.keras.layers.BatchNormalization()(self.encoder_layer)
        self.encoder_layer = tf.keras.layers.LeakyReLU()(self.encoder_layer)
        self.encoder_layer = tf.keras.layers.Dense(units=256)(self.input_layer)
        self.encoder_layer = tf.keras.layers.BatchNormalization()(self.encoder_layer)
        self.encoder_layer = tf.keras.layers.LeakyReLU()(self.encoder_layer)
        self.encoder_layer = tf.keras.layers.Dense(units=128)(self.input_layer)
        self.encoder_layer = tf.keras.layers.BatchNormalization()(self.encoder_layer)
        self.encoder_layer = tf.keras.layers.LeakyReLU()(self.encoder_layer)
        self.z_mean = tf.keras.layers.Dense(self.latent_size)(self.encoder_layer)
        self.z_sigma = tf.keras.layers.Dense(self.latent_size)(self.encoder_layer)
        self.z = tf.keras.layers.Lambda(self.sampling)([self.z_mean, self.z_sigma])
        self.encoder = tf.keras.Model(self.input_layer, [self.z_mean, self.z_sigma, self.z], name='encoder')
        self.latent_inputs = tf.keras.Input(shape=(self.latent_size,), name='z_sampling')
        self.decoder_layer = tf.keras.layers.Dense(units=128)(self.latent_inputs)
        self.decoder_layer = tf.keras.layers.BatchNormalization()(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.LeakyReLU()(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.Dense(units=256)(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.BatchNormalization()(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.LeakyReLU()(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.Dense(units=512)(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.BatchNormalization()(self.decoder_layer)
        self.decoder_layer = tf.keras.layers.LeakyReLU()(self.decoder_layer)
        self.output_layer = tf.keras.layers.Dense(self.input_shape[0], activation='sigmoid')(self.decoder_layer)
        self.decoder = tf.keras.Model(self.latent_inputs, self.output_layer, name='decoder')
        self.output_layer = self.decoder(self.encoder(self.input_layer)[2])
        self.vae = keras.Model(self.input_layer, self.output_layer, name='vae')

    def sampling(self, args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.latentSize),
                              mean=0., stddev=0.1)
        return z_mean + K.exp(z_log_sigma) * epsilon

    def call(self, vector):
        z_mean, z_log_sigma, z = self.encoder(vector)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma) + 1
        )
        self.add_loss(kl_loss)
        self.add_metric(kl_loss, name='kl_loss', aggregation='mean')
        return reconstructed

    def createEmbedding(self, vector):
        return self.encoder(vector)

In [ ]:
dataset = item_attributes[item_attributes.columns[1:]].values

In [ ]:
autoEncoder = VariationalAutoEncoder(inputShape=(dataset.shape[1],),
                                     latentSize=64)
autoEncoder.compile(optimizer=tf.keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError())

2022-11-19 17:31:03.414425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
dataset_tensor = tf.convert_to_tensor(dataset)
dataset_tensor.shape

TensorShape([23691, 904])

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]

In [ ]:
autoEncoder.fit(dataset_tensor, dataset_tensor,
                epochs=200,
                batch_size=32,
                validation_split=0.2,
                callbacks=[callbacks],
                verbose=1)

Epoch 1/200
593/593 [==============================] - 3s 3ms/step - loss: 0.1041 - kl_loss: 0.0815 - val_loss: 0.0180 - val_kl_loss: 0.0032 - lr: 0.0010
Epoch 2/200
593/593 [==============================] - 2s 3ms/step - loss: 0.0190 - kl_loss: 0.0063 - val_loss: 0.0134 - val_kl_loss: 0.0019 - lr: 0.0010
Epoch 3/200
593/593 [==============================] - 2s 3ms/step - loss: 0.0142 - kl_loss: 0.0027 - val_loss: 0.0128 - val_kl_loss: 0.0021 - lr: 0.0010
Epoch 4/200
593/593 [==============================] - 2s 3ms/step - loss: 0.0133 - kl_loss: 0.0024 - val_loss: 0.0123 - val_kl_loss: 0.0022 - lr: 0.0010
Epoch 5/200
593/593 [==============================] - 2s 3ms/step - loss: 0.0130 - kl_loss: 0.0025 - val_loss: 0.0122 - val_kl_loss: 0.0024 - lr: 0.0010
Epoch 6/200
593/593 [==============================] - 2s 3ms/step - loss: 0.0129 - kl_loss: 0.0027 - val_loss: 0.0121 - val_kl_loss: 0.0029 - lr: 0.0010
Epoch 7/200
593/593 [==============================] - 2s 3ms/step - loss: 0

In [ ]:
encoded_dataset = autoEncoder.createEmbedding(dataset_tensor)[2]
#encoded_dataset = np.hstack([encoded_dataset[0], encoded_dataset[1]])
encoded_dataset.shape

TensorShape([23691, 64])

In [ ]:
encoded_dataset

<tf.Tensor: shape=(23691, 64), dtype=float32, numpy=
array([[-0.04274673,  0.04315985,  0.00921491, ..., -0.03659924,
        -0.17490537,  0.16326496],
       [-0.18153004,  0.04759347, -0.00341143, ..., -0.12464795,
        -0.04096986,  0.04406041],
       [-0.05285801, -0.08981786, -0.2105933 , ..., -0.01881974,
        -0.07685136, -0.07413374],
       ...,
       [-0.05543809,  0.00126155, -0.02400142, ...,  0.02111587,
         0.08524208, -0.1505173 ],
       [-0.09645688, -0.0298008 , -0.01686037, ...,  0.07884201,
         0.05439367, -0.02737273],
       [ 0.09505475,  0.07237296,  0.08200808, ...,  0.10972885,
         0.102649  ,  0.40639207]], dtype=float32)>

In [ ]:
np.save('../dataset/ICM', encoded_dataset, allow_pickle=True)